Weather API key = https://home.openweathermap.org/api_keys

In [1]:
!pip install requests llama-index llama-index-llms-groq -q


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Importing the dependencies

In [2]:
import os
import requests
from llama_index.core.agent import ReActAgent
from llama_index.llms.groq import Groq
from llama_index.core.tools import FunctionTool
from dotenv import load_dotenv
load_dotenv()

c:\Users\Admin\Desktop\Siddhardhan Agentic AI - Weather Forecast App with LlamaIndex\Weather Forecast App with LlamaIndex\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


True

**Loading the GroQ LLM**

In [4]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [5]:
llm = Groq(
    model="llama-3.3-70b-versatile",
    temperature=0
)

In [6]:
BASE_URL = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = os.getenv("WEATHER_API_KEY")

In [7]:
city = "Kathmandu"
units = "metric"

In [8]:
params = {
    "q": city,
    "appid": API_KEY,
    "units": units # use "imperial" for Fahrenheit
}

In [9]:
response = requests.get(BASE_URL, params=params)

In [10]:
print(response.json())

{'coord': {'lon': 85.3167, 'lat': 27.7167}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 22, 'feels_like': 21.54, 'temp_min': 22, 'temp_max': 22, 'pressure': 1009, 'humidity': 49, 'sea_level': 1009, 'grnd_level': 850}, 'visibility': 10000, 'wind': {'speed': 2.19, 'deg': 249, 'gust': 2.63}, 'clouds': {'all': 76}, 'dt': 1745133813, 'sys': {'country': 'NP', 'sunrise': 1745106545, 'sunset': 1745153152}, 'timezone': 20700, 'id': 1283240, 'name': 'Kathmandu', 'cod': 200}


**Weather Forecast Function**

In [23]:
import requests

BASE_URL = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = os.getenv("WEATHER_API_KEY")

# custom weather forecast function
def find_weather(city: str) -> dict:
  """This function returns the current weatehr forecast for the given city"""
  units = "metric"
  params = {
      'q': city,
      'appid': API_KEY,
      'units': units
  }
  response = requests.get(BASE_URL, params=params)
  result = response.json()

  return result

In [24]:
# convert the function to a function tool
weather_tool = FunctionTool.from_defaults(fn=find_weather)

In [25]:
tools_list = [weather_tool]

In [26]:
agent = ReActAgent.from_tools(
    tools = tools_list,
    llm = llm,
    verbose = True
)

In [27]:
response = agent.chat("What is the temperature like in Kathmandu?")

> Running step 908e5581-a38c-4f26-87a2-7395550ea932. Step input: What is the temperature like in Kathmandu?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: find_weather
Action Input: {'city': 'Kathmandu'}
Observation: {'coord': {'lon': 85.3167, 'lat': 27.7167}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 22, 'feels_like': 21.54, 'temp_min': 22, 'temp_max': 22, 'pressure': 1009, 'humidity': 49, 'sea_level': 1009, 'grnd_level': 850}, 'visibility': 10000, 'wind': {'speed': 2.19, 'deg': 249, 'gust': 2.63}, 'clouds': {'all': 76}, 'dt': 1745134016, 'sys': {'country': 'NP', 'sunrise': 1745106545, 'sunset': 1745153152}, 'timezone': 20700, 'id': 1283240, 'name': 'Kathmandu', 'cod': 200}
> Running step da8d954e-f9b7-4b21-8d0e-f0ffe30034a1. Step input: None
Thought: I have the weather information for Kathmandu. The temperature is 22 degrees Celsiu

In [28]:
print(response.response)

The temperature in Kathmandu is 22 degrees Celsius.
